In [1]:
from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np

from ta.trend import trix, sma_indicator
from ta.momentum import rsi, stochrsi
from modules.backtester import long_only_backtester
from modules.data_fetcher import download_historical_data

In [5]:
symbol = "BTC-USDT"
trix_length = 9
trix_signal = 21
rsi_window = 14

df_BTC = download_historical_data(symbol, "4hour")  # .loc["2021-11-20":]
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

(11350, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2018-01-01 01:00:00,1.514765e+09,12933.000001,13000.000000,13889.000000,12510.000000,5.534502,72706.427555
2018-01-01 05:00:00,1.514779e+09,13000.000000,13300.000000,13446.510264,12600.000001,5.281884,69154.117003
2018-01-01 09:00:00,1.514794e+09,13697.624103,13839.999981,13989.999997,13050.000000,2.927550,38829.908810
2018-01-01 13:00:00,1.514808e+09,13001.000000,13221.999990,13839.999982,12500.000000,5.069823,64820.202765
2018-01-01 17:00:00,1.514822e+09,13000.000001,13540.443994,13799.999998,13000.000000,4.926572,65319.109039


In [6]:
df_BTC["Trix"] = trix(df_BTC.Close, trix_length)

trix_pct = df_BTC["Trix"].pct_change() * 100
trix_sig = sma_indicator(trix_pct, trix_signal)
df_BTC["Trix_histo"] = trix_pct - trix_sig

df_BTC['Stoch_RSI'] = stochrsi(df_BTC.Close, rsi_window)

df_BTC.dropna(inplace=True)

df_BTC.head(5)

,Timestamp,Open,Close,High,Low,Amount,Volume,Trix,Trix_histo,Stoch_RSI
Date,,,,,,,,,,
2018-01-08 17:00:00,1.515427e+09,13619.57800,15200.000000,15499.000000,13619.578000,60.511958,871383.049224,-0.208000,161.943543,0.429401
2018-01-08 21:00:00,1.515442e+09,15100.00000,15091.910250,15499.000000,14499.800000,26.035242,387149.144214,-0.299695,47.274057,0.402968
2018-01-09 01:00:00,1.515456e+09,15077.71860,15230.000000,15850.000000,14858.277611,22.596351,342880.239185,-0.354725,21.789710,0.443943
2018-01-09 05:00:00,1.515470e+09,15230.00000,14989.990000,15799.669768,14535.000000,28.992772,438004.243716,-0.394634,15.039592,0.412888
2018-01-09 09:00:00,1.515485e+09,14989.98999,14710.000002,15497.229995,14036.000000,34.567751,510626.603973,-0.433840,14.388102,0.335187


In [7]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.Trix_histo > 0  and row.Stoch_RSI < 0.8 else False


def sell_func(row: pd.Series, prev_row: pd.Series, timeframe_count: int) -> bool:
    return True if row.Trix_histo < 0  and row.Stoch_RSI > 0.2 else False


long_only_backtester(df_BTC, buy_func, sell_func)

-------------  General informations  -------------
Period: [2018-01-08 17:00:00] -> [2023-03-07 13:00:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 1349.51 $
Final net balance: 418.98 $
Strategy net return: 41.90 %
Buy and Hold return: 147.36 %
Strategy winrate: 54.79 %
Strategy fees: 1042.31 $
Strategy volatility: 0.04 %
Sharpe ratio: 0.03 (no risk free rate)
Sharpe ratio: 0.03 (risk free rate = buy and hold)

-------------  Trades informations  --------------
Mean trade duration: 1 days 06:24:39
Total trades: 584
Total good trades: 320
Mean good trades return: 2.52 %
Median good trades return: 1.32 %
Best trades return: 18.69 % | Date: 2018-04-13 10:00:00 | Duration: 1 days 04:00:00
Mean good trade duration: 1 days 02:47:03

Total bad trades: 264
Mean bad trades return: -2.73 %
Median bad trades return: -1.35 %
Worst trades return: -29.58 % | Date: 2020-03-13 13:00:00 | Duration: 1 days 08:00:00
Mean bad trade duration: 1 days 10:48:24
